In [4]:
# Text entailment predictor
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/esim-elmo-2018.05.17.tar.gz", 'textual-entailment')

W0527 01:56:32.544157 140736874374080 file_utils.py:35] Deprecated cache directory found (/Users/linzhijia/.allennlp/datasets).  Please remove this directory from your system to free up space.
/Users/linzhijia/Library/Python/3.6/lib/python/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [1]:
import numpy as np
import pandas as pd
import json
import os
import csv
import time
import nltk
from nltk import word_tokenize
from nltk.corpus import wordnet
import re
from collections import Counter
from collections import defaultdict
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download("wordnet")
import unicodedata
from nltk.corpus import stopwords
import string
from math import log, sqrt
import spacy
from spacy import displacy
# Stopwords
stop = stopwords.words('english') + list(string.punctuation)+list(('``','--'))+list(("-lrb-", "-rrb-"))
nlp = spacy.load('en_core_web_md')

[nltk_data] Downloading package wordnet to /home/nizam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


OSError: [E050] Can't find model 'en_core_web_md'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

# 1. Create inverted index

In [1]:
import time
filenames = []
for i in range(1,10):
    filenames.append('wiki-pages-text/wiki-00'+ str(i) + '.txt')
for i in range(10,100):
    filenames.append('wiki-pages-text/wiki-0'+ str(i)+'.txt')
for i in range(100,110):
    filenames.append('wiki-pages-text/wiki-'+ str(i)+'.txt')

file_as_list = []#contains a file read from wiki-pages-text
#wiki_sentences = []
all_wiki_sentences = []
identifier = []
st = time.time()
files_processed = 0
#term_id = -1 # global variable for assigning unique id to word types(vocabulary)
for filename in filenames:#looping 109 files
    with open(filename) as wikifile:#opening 1 file at a time
        file_as_list.append(wikifile.readlines())#reading 1 file into a list
    for file in file_as_list:
        for sent in file:
            page_id,sent_num,sent = sent.split(" ",2)
            #wiki_sentences.append(sent)
            all_wiki_sentences.append(sent)
            identifier.append((page_id,sent_num))
    #write_to_file(processed_sents,uid)
    #write_to_file(wiki_sentences)
    file_as_list = []
    #wiki_sentences = []
    files_processed += 1
et = time.time()
print("files_processed: ", files_processed)
print("Time for processing wiki-corpus:",(et-st)/(3600.0),"hrs.")

FileNotFoundError: [Errno 2] No such file or directory: 'wiki-pages-text/wiki-001.txt'

In [ ]:
st = time.time()  
processed_sents = []
# vocab contains (term, term id) pairs
#vocab = {}
# total_tokens stores the total number of tokens
total_tokens = 0
# total_stems stores the total number of terms in vocab
#total_lemmas = 0

for sent in all_wiki_sentences:
    norm_sent = preprocessed_sentence(sent)
    for token in norm_sent:
        total_tokens += 1
        #if token not in vocab:
            #vocab[token] = total_lemmas
            #total_lemmas += 1
    processed_sents.append(norm_sent)  
et = time.time()   
print("Time for processing wiki-corpus:",(et-st)/(3600.0),"hrs.")
print("Number of documents = {}".format(len(processed_sents)))
#print("Number of unique terms = {}".format(len(vocab)))
print("Number of tokens = {}".format(total_tokens))

In [ ]:
# doc_term_freqs stores the counters (mapping terms to term frequencies) of all documents
doc_term_freqs = []
st = time.time()
for tokens in processed_sents:
    tfs = Counter()
    for token in tokens:
        tfs[token] += 1
    doc_term_freqs.append(tfs)
et = time.time()
print("Time used: ",(et - st)/60.0," mins") 
print(len(doc_term_freqs))
print(doc_term_freqs[0])
print(doc_term_freqs[-109])

In [7]:
class InvertedIndex:
    def __init__(self, vocab, doc_term_freqs):
        self.vocab = vocab
        self.doc_len = [0] * len(doc_term_freqs)
        self.doc_term_freqs = [[] for i in range(len(vocab))]
        self.doc_ids = [[] for i in range(len(vocab))]
        self.doc_freqs = [0] * len(vocab)
        self.total_num_docs = 0
        self.max_doc_len = 0
        for docid, term_freqs in enumerate(doc_term_freqs):
            doc_len = sum(term_freqs.values())
            self.max_doc_len = max(doc_len, self.max_doc_len)
            self.doc_len[docid] = doc_len
            self.total_num_docs += 1
            for term, freq in term_freqs.items():
                try:
                    term_id = vocab[term]
                    self.doc_ids[term_id].append(docid)
                    self.doc_term_freqs[term_id].append(freq)
                    self.doc_freqs[term_id] += 1
                except KeyError:
                    continue

    def num_terms(self):
        return len(self.doc_ids)

    def num_docs(self):
        return self.total_num_docs

    def docids(self, term):
        term_id = self.vocab[term]
        return self.doc_ids[term_id]

    def freqs(self, term):
        term_id = self.vocab[term]
        return self.doc_term_freqs[term_id]

    def f_t(self, term):
        term_id = self.vocab[term]
        return self.doc_freqs[term_id]

    def space_in_bytes(self):
        # this function assumes each integer is stored using 8 bytes
        space_usage = 0
        for doc_list in self.doc_ids:
            space_usage += len(doc_list) * 8
        for freq_list in self.doc_term_freqs:
            space_usage += len(freq_list) * 8
        return space_usage
    
invindex = InvertedIndex(vocab, doc_term_freqs)

In [ ]:
import pickle
def save_object(obj, filepath):
    '''
    function to save a pickle object
    '''
    with open(filepath, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

# Save the inverted index for future use
save_object(invindex, 'invindex_final.pkl')

def load_object(filepath):
    '''
    function to load a pickle object
    '''
    obj=None
    with (open(filepath, "rb")) as openfile:
        while True:
            try:
                obj = pickle.load(openfile)
            except EOFError:
                break
    return obj

# Re-load the index
invindex=load_object('invindex_final.pkl')               

### Main method

In [ ]:
import preprocess
import misc
doc_term_freq,vocab=create_doc_term_freq(filepath)
invindex = InvertedIndex(vocab, doc_term_freqs)
save_object(invindex, 'invindex_final.pkl')
#invindex=load_object('invindex_final.pkl')               

# 2. Sentences Retrival

In [2]:
def lemmatize_my_token(word):
    lemmatizer = WordNetLemmatizer()
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma
def preprocessed_sentence(sent):
    norm_sent = []#contains normalized tokens of a single sentence(treated as a document)
    for token in nltk.word_tokenize(sent.lower()):#1.all tokens lowercased
        nfd_token = unicodedata.normalize('NFD',token)#2.NFD normalization applied
        if nfd_token not in stop:#3.stop words and punctuations excluded
            #norm_token = lemmatize_my_token(nfd_token)#4.lemmatized
            #norm_sent.append(norm_token)
            norm_sent.append(nfd_token)
    return norm_sent

# 2.1 Using bm25 ro retrieve sentences

In [11]:
def extract_term_freqs(preprocessed_sent):
    tfs = Counter()    
    for token in preprocessed_sent:
        tfs[token] += 1
    return tfs

#compute BM25
N = invindex.num_docs()
Lavg = sum(invindex.doc_len)/N

# query: a claim index: the inverted index   k: the maximun number of sentences returned
def bm_25(query, index, k):      
    
    scores_bm25 = Counter()
    #scores_tfidf = Counter()
    query_terms = preprocessed_sentence(query)
    query_terms_freqs = extract_term_freqs(query_terms)
    k1 = 1.2
    k3 = 1.5
    b = 0.75
    
    def fill_scores_one(term):
        try:
            f_term = index.f_t(str(term))         #number of docs containing the term 
            docids = index.docids(str(term))    #list of Ids of the documents containing the term 't'
            fqt = query_terms_freqs[str(term)]   # number of term in a query
            
            
            def fill_scores_two(d, docid): 
           
            #for d,docid in enumerate(docids):   
            
                fdt =  index.freqs(term)[d]  # number of a term in a sentence
                Ld = index.doc_len[docid]    # length of a sentence
                idf = log((N - f_term + 0.5)/(f_term + 0.5))
                tf_doc = ((k1 + 1) * fdt)/(k1 * ((1-b) + b * Ld/Lavg) + fdt)
                query_tf = (k3 + 1) * fqt / (k3 + fqt)
                wt = idf * tf_doc * query_tf
                scores_bm25[docid] += wt
            
            temp = [*map(fill_scores_two, list(range(len(docids))), docids)]
            temp = []
             
                #temp = 0 #an auxiliary variable to aid in computing the score.
                #temp = temp + log(1 + fdt) * log(N/float(f_term))
                #scores_tfidf[docid] += 1.0/sqrt(index.doc_len[docid]) * temp
        except KeyError:
            pass
        except IndexError:
            print("Index error occured. Try again later.")

    
    dummy = [*map(fill_scores_one, query_terms)]
    dummy = []
    
    #final_docids = set()
    #for x in scores_bm25.most_common(k):
        #final_docids.add(x[0])
    #for x in scores_tfidf.most_common(k):
        #final_docids.add(x[0])
    return [x[0] for x in scores_bm25.most_common(k)]

# 2.2 Using the entities in the claim to retrive sentences

In [ ]:
# Create a dictionary of processed identifiers
proccessed_ids = defaultdict(list)
for doc_id, pair in enumerate(identifier):
    proccessed_id = unicodedata.normalize('NFD',re.sub('_',' ', re.sub('-(\w)+-','', pair[0])).lower())
    proccessed_id[proccessed_id].append((pair[1], doc_id))

In [181]:
# Types of the entities in the claim
type_words = ['film', 'movie', 'song', 'band', 'television series', 'novel', \
              'comics', 'magazine', 'play', 'soundtrack']

def find_ents_ids(query):
    
    query = unicodedata.normalize('NFD',query)
    evidence_ids = []
    
    # Search possible type words
    query_type = None
    for type_word in type_words:
        if type_word in query.lower():
            if type_word == 'movie':
                query_type = 'film'
                break
            if type_word == 'television series':
                query_type = 'tv series'
                break
            query_type = type_word
            break   
    
    # Find capitaltized tokens
    temp = []
    num_cap = 0
    cap_token = []
    tokens = word_tokenize(query)
    length_query = len(tokens)
    i = 0
    for token in tokens:
        i += 1
        if token[0].isupper():
            num_cap += 1
            cap_token.append(token)
            if i == length_query:
                temp.append(' '.join(cap_token))
        else:
            if num_cap == 0:
                continue
            else:
                if cap_token[-1] in ['of', 'a', 'the']:
                    cap_token.append(token)
                elif token in ['of', 'a', 'the'] and token != cap_token[-1]:
                    cap_token.append(token)
                elif token.isnumeric():
                    cap_token.append(token)  
                else:
                    if num_cap > 0:
                        temp.append(' '.join(cap_token))
                        num_cap = 0
                        cap_token = []

    cap_tokens = [re.sub('the ', '', token.lower()) for token in temp]
    #print(cap_tokens)
    
    def find_matching(item):
        return list(filter(lambda x:item in x, list(proccessed_ids.keys())))

    # Find identifiers that contain the capitaltized tokens
    ids_list = [*map(find_matching, cap_tokens)]
    i = 0
    for ids in ids_list:
        if len(ids) <= 3:
            evidence_ids.extend(ids)
        else:
            if query_type != None:
                item = cap_tokens[i]+" "+query_type
                evidence_ids.extend(list(filter(lambda x: item in x, ids)))  
            else:
                if len(evidence_ids) == 0:
                    item = cap_tokens[i]
                    evidence_ids.extend(list(filter(lambda x : x == item, ids)))
        i += 1
    return evidence_ids

#  Find all document ids for all identifiers
def find_ents_sentences(query):
    ids = find_ents_ids(query)
    evidence_docids = []
    for id in ids:
        for _,doc_id in proccessed_ids[id]:
            evidence_docids.append(doc_id)
    return evidence_docids

In [14]:
# Combine the result of the two methods
def find_possible_sentences(query, k):    
    results_bm25 = bm_25(query, invindex, k)
    results_ents = find_ents_sentences(query)
    sents = set()
    for res in results_bm25:
        sents.add((res, identifier[res]))
    for res in results_ents:
        sents.add((res, identifier[res]))
    return sents

# 3.1.1 Function for Generating 3 Sentence for a training data

<b>Selection of training data set</b>
<b>1.</b>Select only those training records where length of evidences is atmost 25.
<b>2.</b>Randomly Select 100 if 'SUPPORT' labelled or 200 if 'REFUTE' labelled from each set of records whose number of evidences whose length is between 1 and 25;
if the length of evidences is less than 100 then select all.
<b>3.</b>Therefore, 100 x 25 = 2500 records from support, refute and random 2500 from not enough case since
when dealing with not enough case we do not have any evidences in training data.

In [ ]:
import json
with open('train.json') as trainfile:
    train_data = json.load(trainfile)
training_support_data = []
training_refute_data = []
training_notenough_data = []
for record in train_data:
    if train_data[record]['label']=='SUPPORTS':
        training_support_data.append(train_data[record])
    elif train_data[record]['label']=='REFUTES':
        training_refute_data.append(train_data[record])
    elif train_data[record]['label']=='NOT ENOUGH INFO':
        training_notenough_data.append(train_data[record])
assert(len(train_data)==len(training_support_data)+len(training_refute_data)+len(training_notenough_data))
print("--------------------Distribution in training dataset-----------------------------------------")
print('Percent of support entries %.3f %%'%(100*(len(training_support_data)/len(train_data))))
print('Percent of refute entries %.3f %%'%(100*(len(training_refute_data)/len(train_data))))
print('Percent of not-enough entries %.3f %%'%(100*(len(training_notenough_data)/len(train_data))))
print("---------------------------------------------------------------------------------------------")
#1.
temp=[]
temp2={}
refute = {}
support = {}
MAX_LENGTH_OF_EVIDENCES = 25
for record in training_refute_data:
    temp.append(len(record['evidence']))
for i in range(1,MAX_LENGTH_OF_EVIDENCES+1):#Note:the max number of evidences for SUPPORT/REFUTE is 48
    temp2[i]=0
temp = sorted(temp)
temp = temp[0:temp.index(MAX_LENGTH_OF_EVIDENCES+1)]
for i in temp:
    temp2[i]+=1
refute =temp2
#refute is a dictionary where key=length of evidences and value=no of evidences whose length is as indicated by key
#------------------------------------------------------------------------------------------------------------
temp=[]
temp2={}
for record in training_support_data:
    temp.append(len(record['evidence']))
for i in range(1,MAX_LENGTH_OF_EVIDENCES+1):#Note:the max number of evidences for SUPPORT/REFUTE is 48
    temp2[i]=0
temp = sorted(temp)
temp = temp[0:temp.index(MAX_LENGTH_OF_EVIDENCES+1)]
for i in temp:
    temp2[i]+=1
support=temp2
#support is a dictionary where key=length of evidences and value=no of evidences whose length is as indicated by key
#2
import numpy as np
def get_indices_and_shuffle(dataset):
    length_indicies = {}#dictionary...key=length of evidences value:list of indices
    for i in list(range(1,MAX_LENGTH_OF_EVIDENCES+1)):
        temp=list()
        length_indicies[i]=temp
    for record in dataset:
        if len(record['evidence']) in list(range(1,26)):
            temp_list = length_indicies[len(record['evidence'])]
            temp_list.append(dataset.index(record))
    return length_indicies
length_indicies_support = get_indices_and_shuffle(training_support_data)
length_indicies_refute = get_indices_and_shuffle(training_refute_data)
sample_not_enough = random.sample(training_notenough_data,2500)
import random
def shuffle_indices(length_indicies_dict,k):
    final_indices = []
    for length,index_list in length_indicies_dict.items():
        if(len(index_list)<=k):
            final_indices.extend(index_list)
        else:
            final_indices.extend(random.sample(index_list, k))
    if(len(final_indices)<2500):
        final_indices.extend(random.sample(length_indicies_dict[3],2500-len(final_indices)))
        final_indices = set(final_indices)
    np.random.shuffle(list(final_indices))
    return final_indices
refute_indices  = shuffle_indices(length_indicies_refute,200)
support_indices = shuffle_indices(length_indicies_support,100)
training_sample = []
for index in refute_indices:
    training_sample.append(training_refute_data[index])
for index in support_indices:
    training_sample.append(training_support_data[index])
training_sample.extend(sample_not_enough)
np.random.shuffle(training_sample)
assert(len(training_sample)==len(refute_indices)+len(support_indices)+len(sample_not_enough))

In [558]:
def get_evidence_docid(evidence):
    evidence_id = unicodedata.normalize('NFD',re.sub('_',' ', re.sub('-(\w)+-','', evidence[0])).lower())
    evidence_sent_num = str(evidence[1])
    for sent_num, docid in proccessed_ids[evidence_id]:
        if evidence_sent_num == sent_num:
            return docid
            break

def get_evidence_output(docid, query):
    
    norm_id = unicodedata.normalize('NFD',re.sub('_',' ', re.sub('-LRB-(\w)+-RRB-','', identifier[docid][0])))
    sent = clean_sent(all_wiki_sentences[docid])
    for word in pronouns_one:
        if word in sent:
            sent = re.sub(word, ' '+ norm_id + ' ', sent)
    for word in pronouns_two:
        if word in sent:
            sent = re.sub(word, ' ' + norm_id + '\'s ', sent)    
    sent = re.sub('.\n','',sent)
    #print(sent)
    
    sent_embedding = session.run(embedded_text, feed_dict={text_input: [sent]})
    query_embedding = session.run(embedded_text, feed_dict={text_input: [query]})
    sim = list(cosine_similarity(sent_embedding, query_embedding).flatten())
    score = get_label_scores(query, sent)
    #print(score)
    return (sim[0], score[0], score[1], score[2])

N = invindex.num_docs()
def get_training_data_for_sample(sample):
    LIMIT = 3
    if sample['label'] == 'NOT ENOUGH INFO':
        output = label_and_evidenve(sample['claim'] , LIMIT)
    else:
        docids = [get_evidence_docid(evidence) for evidence in sample['evidence'][:LIMIT]]
        if len(docids) == 3:
            output = [*map(lambda docid:(get_evidence_output(docid, sample['claim'])), docids)]
        else:
            output_a = [*map(lambda docid:(get_evidence_output(docid, sample['claim'])), docids)]
            if len(docids) == 1:
                if docids[0] + 2 <= N - 1:
                    output_b = [*map(lambda k:(get_evidence_output(docids[0] + k, sample['claim'])), list(range(1,3)))]
                    output = output_a + output_b 
                else:
                    output_b = [*map(lambda k:(get_evidence_output(docids[0] - k, sample['claim'])), list(range(1,3)))]
                    output = output_a + output_b 
            if len(docids) == 2:
                if docids[0] + 1 <= N - 1:
                    output_b = [get_evidence_output(docids[0] + 1, sample['claim'])]
                    output = output_a + output_b
                else:
                    output_b = [get_evidence_output(docids[0] - 1, sample['claim'])]
                    output = output_a + output_b

#             if len(docids) == 3:
#                 output_b = [*map(lambda docid:(get_evidence_output(docid + 1, sample['claim'])), docids)]
#             if len(docids) == 4:
#                 output_b = [get_evidence_output(docids[0] + 1, sample['claim'])]
#             output = output_a + output_b 
                
    return output[:LIMIT]                 

 # 3.1.2 Function for Generating 4 Sentence for a training data

In [632]:
def get_evidence_docid(evidence):
    evidence_id = unicodedata.normalize('NFD',re.sub('_',' ', re.sub('-(\w)+-','', evidence[0])).lower())
    evidence_sent_num = str(evidence[1])
    for sent_num, docid in proccessed_ids[evidence_id]:
        if evidence_sent_num == sent_num:
            return docid
            break

# Generating 
def get_evidence_output(docid, query):
    
    norm_id = unicodedata.normalize('NFD',re.sub('_',' ', re.sub('-LRB-(\w)+-RRB-','', identifier[docid][0])))
    sent = clean_sent(all_wiki_sentences[docid])
    for word in pronouns_one:
        if word in sent:
            sent = re.sub(word, ' '+ norm_id + ' ', sent)
    for word in pronouns_two:
        if word in sent:
            sent = re.sub(word, ' ' + norm_id + '\'s ', sent)    
    sent = re.sub('.\n','',sent)
    #print(sent)
    
    sent_embedding = session.run(embedded_text, feed_dict={text_input: [sent]})
    query_embedding = session.run(embedded_text, feed_dict={text_input: [query]})
    sim = list(cosine_similarity(sent_embedding, query_embedding).flatten())
    score = get_label_scores(query, sent)
    #print(score)
    return (sim[0], score[0], score[1], score[2])

def get_training_data_for_sample(sample):
    LIMIT = 4
    if sample['label'] == 'NOT ENOUGH INFO':
        output = label_and_evidenve(sample['claim'] , LIMIT)
    else:
        docids = [get_evidence_docid(evidence) for evidence in sample['evidence'][:LIMIT]]
        if len(docids) == 4:
            output = [*map(lambda docid:(get_evidence_output(docid, sample['claim'])), docids)]
        else:
            output_a = [*map(lambda docid:(get_evidence_output(docid, sample['claim'])), docids)]
            if len(docids) == 1:
                output_b = [*map(lambda k:(get_evidence_output(docids[0] + k, sample['claim'])), list(range(1,4)))]
                output = output_a + output_b 
            if len(docids) == 2:
                output_b = [*map(lambda docid:(get_evidence_output(docid + 1, sample['claim'])), docids)]
                output = output_a + output_b 
            if len(docids) == 3:
                output_b = [get_evidence_output(docids[0] + 1, sample['claim'])]
                output = output_a + output_b
                
    return output  

# 3.2 Using SNLI and training data to train the model

In [ ]:
# fit mlp model on problem 2 and save model to file
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical
from matplotlib import pyplot
import category_encoders as ce

In [ ]:
df_train = pd.read_csv('snli_1.0_train.csv')
df_train = df_train.drop(set(list(df_train.columns))-set(['gold_label','sentence1','sentence2']),axis=1)
df_train=df_train.iloc[:1000]
df_train.columns = ['label','claim','evidence']
claim_evidence = []
labels = []
for index, row in df_train.iterrows():
    labels.append((row['label']))
    claim_evidence.append(((row['claim'],row['evidence'])))

In [265]:
def get_output_for_snli_data(hyp, premise):
    hyp_embedding = session.run(embedded_text, feed_dict= {text_input: [hyp]})
    premise_embedding = session.run(embedded_text, feed_dict={text_input: [premise]})
    sim = list(cosine_similarity(hyp_embedding, premise_embedding).flatten())
    score = get_label_scores(hyp, premise)
    #print(score)
    return (sim[0], score[0], score[1], score[2])

In [831]:
df_train = pd.read_csv('snli_1.0_train.csv')
df_train = df_train.drop(set(list(df_train.columns))-set(['gold_label','sentence1','sentence2']),axis=1)
df_train = df_train.iloc[999:9999]
df_train.columns = ['label','claim','evidence']
claim_evidence_train_large = []
labels_train_large = []
for index, row in df_train.iterrows():
    labels_train_large.append((row['label']))
    claim_evidence_train_large.append(((row['claim'],row['evidence'])))
snli_output_train_large = []
for hyp, premise in claim_evidence_train_large:
    snli_output_train_large.append(get_output_for_snli_data(hyp, premise))

In [ ]:
df_dev = pd.read_csv('snli_1.0_dev.csv')
df_dev = df_dev.drop(set(list(df_dev.columns))-set(['gold_label','sentence1','sentence2']),axis=1)
df_dev=df_dev.iloc[:300]
df_dev.columns = ['label','claim','evidence']
claim_evidence_dev = []
labels_dev = []
for index, row in df_dev.iterrows():
    labels_dev.append((row['label']))
    claim_evidence_dev.append(((row['claim'],row['evidence'])))
snli_output_dev = []
for hyp, premise in claim_evidence_dev:
    snli_output_dev.append(get_output_for_snli_data(hyp, premise))

In [ ]:
train_X_large = pd.DataFrame(snli_output_train_large)# TRAINX is a list
train_y_large = pd.DataFrame(labels_train_large)#TRAINY IS A LIST
train_y_large = temp.fit_transform(train_y_large)
train_y_large.columns = ['SUPPORTS','REFUTES','NOT_ENOUGH_INFO']

In [ ]:
#loading training and development datasamples into dataframes
train_X = pd.DataFrame(snli_output_train_large)# TRAINX is a list
train_y = pd.DataFrame(labels_train_large)#TRAINY IS A LIST
dev_X = pd.DataFrame(snli_output_dev)# DEVX is a list
dev_y = pd.DataFrame(labels_dev)# DEVY is a list
#ENCODING our labels as one hot encoded
temp = ce.OneHotEncoder()
train_y = temp.fit_transform(train_y)
train_y.columns = ['REFUTES','SUPPORTS','NOT_ENOUGH_INFO']#WARNING: naming is done by looking at dataframes manually to be consistent!!!!
dev_y = temp.fit_transform(dev_y)
dev_y.columns = ['NOT_ENOUGH_INFO','REFUTES','SUPPORTS']#WARNING: naming is done by looking at dataframes manually to be consistent!!!!

In [ ]:
#Transform training data

%%time
failure = []
for sample in train_sample:
    try:
        training_data.append(get_training_data_for_sample(sample))
    except:
        failure.append(i)

In [639]:
#1172 in training_sample has problrm
training_data_labels_2000 = []
for sample in train_sample[:1172]:
    if sample['label'] == 'NOT ENOUGH INFO':
        training_data_labels_2000 .extend(['NOT ENOUGH INFO']*3)
    else:
        evi_len = len(sample['evidence']) 
        if evi_len >= 3:
            training_data_labels_2000 .extend([sample['label']]*3)
        else:
            training_data_labels_2000 .extend([sample['label']]*evi_len)
            training_data_labels_2000 .extend(['NOT ENOUGH INFO']*(3 - evi_len))
for sample in train_sample[1173:2000]:
    if sample['label'] == 'NOT ENOUGH INFO':
        training_data_labels_2000 .extend(['NOT ENOUGH INFO']*3)
    else:
        evi_len = len(sample['evidence']) 
        if evi_len >= 3:
            training_data_labels_2000 .extend([sample['label']]*3)
        else:
            training_data_labels_2000 .extend([sample['label']]*evi_len)
            training_data_labels_2000 .extend(['NOT ENOUGH INFO']*(3 - evi_len))

In [ ]:
loading training and development datasamples into dataframes
train_X = pd.DataFrame(train_data)# TRAINX is a list
train_y = pd.DataFrame(training_data_labels_2000)#TRAINY IS A LIST
dev_X = pd.DataFrame(snli_output_dev)# DEVX is a list
dev_y = pd.DataFrame(labels_dev)# DEVY is a list
#ENCODING our labels as one hot encoded
temp = ce.OneHotEncoder()
train_y = temp.fit_transform(train_y)
train_y.columns = ['NOT_ENOUGH_INFO','REFUTES','SUPPORTS']#WARNING: naming is done by looking at dataframes manually to be consistent!!!!
dev_y = temp.fit_transform(dev_y)
dev_y.columns = ['NOT_ENOUGH_INFO','SUPPORTS', 'REFUTES']#WARNING: naming is done by looking at dataframes manually to be consistent!!!!

# 3.3 Fit and re-fit model

In [ ]:
# Reference: https://machinelearningmastery.com/

# define and fit model on a training dataset
def fit_model(trainX, trainy, devX, devy):
    # define model
    model = Sequential()
    model.add(Dense(5, input_dim = 4, activation='relu',use_bias=True, kernel_initializer='he_uniform'))#Hidden layer with 5 units
    #'he_uniform' draws inital weights from uniform distribution
    model.add(Dense(3, activation='softmax'))
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit model
    history = model.fit(trainX, trainy, validation_data=(devX, devy), epochs=100, verbose=0,batch_size=128)
    return model, history

# summarize the performance of the fit model
def summarize_model(model, history, trainX, trainy, devX, devy):
    # evaluate the model
    _, train_acc = model.evaluate(trainX, trainy, verbose=0)
    _, dev_acc = model.evaluate(devX, devy, verbose=0)
    print('Train: %.3f, Test: %.3f' % (train_acc, dev_acc))
    # plot loss during training
    pyplot.subplot(211)
    pyplot.title('Loss')
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='dev')
    pyplot.legend()
    # plot accuracy during training
    pyplot.subplot(212)
    pyplot.title('Accuracy')
    pyplot.plot(history.history['acc'], label='train')
    pyplot.plot(history.history['val_acc'], label='dev')
    pyplot.legend()
    pyplot.show()


In [ ]:
# fit model on train dataset
model, history = fit_model(train_X, train_y, dev_X, dev_y)
# evaluate model behavior
summarize_model(model, history, trainX, trainy, devX, devy)
#!python3 -m pip install h5py --user
model.save('model_snli.h5')

In [806]:
from keras.models import load_model
# define and fit model on a training dataset
def refit_model(trainX, trainy, devX, devy):
    # load the model we had stored earlier
    # model trainded by 2000 claims in the traing data
    model = load_model('model_snli_large2.h5')

    # compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit model
    history = model.fit(trainX, trainy, validation_data=(devX, devy), epochs=100, verbose=0,batch_size=128)
    return model, history

# summarize the performance of the fit model
def summarize_model(model, history, trainX, trainy, devX, devy):
    # evaluate the model
    _, train_acc = model.evaluate(trainX, trainy, verbose=0)
    _, dev_acc = model.evaluate(devX, devy, verbose=0)
    print('Train: %.3f, Test: %.3f' % (train_acc, dev_acc))
    # plot loss during training
    pyplot.subplot(211)
    pyplot.title('Loss')
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='dev')
    pyplot.legend()
    # plot accuracy during training
    pyplot.subplot(212)
    pyplot.title('Accuracy')
    pyplot.plot(history.history['acc'], label='train')
    pyplot.plot(history.history['val_acc'], label='dev')
    pyplot.legend()
    pyplot.show()
  
  
# refit model on train dataset
robust_model, history = refit_model(train_X, train_y, dev_x_0to500, dev_y_0to500)
# evaluate model behavior
summarize_model(robust_model, history, train_X, train_y, dev_x_0to500, dev_y_0to500)
#!python3 -m pip install h5py --user
model_train_0to2000.h5
#robust_model.save('model_train_0to2000.h5')#OUR FINAL MODEL FOR PREDICTING TEST DATASET

# 4 Prediction

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity

#embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")

W0527 01:26:24.458389 140736874374080 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [3]:
# Create graph and finalize (finalizing optional but recommended).
g = tf.Graph()
with g.as_default():
    text_input = tf.placeholder(dtype=tf.string, shape=[None])
    sent_embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")
    embedded_text = sent_embed(text_input)
    init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
    g.finalize()

# Create session and initialize.
session = tf.Session(graph=g)
session.run(init_op)

W0527 01:47:04.355772 140736874374080 resolver.py:330] Deleting lock file /var/folders/nm/11p2hm4d6mq71ld5bn3cmpjh0000gn/T/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47.lock due to inactivity.


Instructions for updating:
Colocations handled automatically by placer.


W0527 01:56:13.592516 140736874374080 deprecation.py:323] From /Users/linzhijia/Library/Python/3.6/lib/python/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0527 01:56:15.254437 140736874374080 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [ ]:
# Using entaiment predicttor to get three probablities
def get_label_scores(query, sentence):
    return predictor.predict(hypothesis = query,premise = sentence)['label_probs']

def get_label(scores):
    sup, ref, noI = scores
    if max(sup, ref, noI) == sup:
        return "SUPPORTS"
    if max(sup, ref, noI) == ref:
        return "REFUTES"
    if max(sup, ref, noI) == noI:
        return "NOT ENOUGH INFO"

In [721]:
pronouns_one = [' he ', 'He ', 'She ', ' she ', 'It ', ' it ']
pronouns_two = [' his ', 'His ', ' her ', 'Her ', ' its ', 'Its ']

def sort(tup): 
    return(sorted(tup, key = lambda x: float(x[0]), reverse = True))

def clean_sent(sent):
    return re.sub('\'', '', re.sub( " -RRB-",')', re.sub("-LRB- ",'(',re.sub(" $" , '', re.sub('.\n','', sent)))))

scores_list = []
def label_and_evidenve(query, k):

    result = find_possible_sentences(query, 5)
    
    messages = []
    ids = []
    
    
    for docid,pair in result:
        norm_id = unicodedata.normalize('NFD',re.sub('_',' ', re.sub('-LRB-(\w)+-RRB-','', pair[0])))
        sent = clean_sent(all_wiki_sentences[docid])
        for word in pronouns_one:
            if word in sent:
                sent = re.sub(word, ' '+ norm_id + ' ', sent)
        for word in pronouns_two:
            if word in sent:
                sent = re.sub(word, ' ' + norm_id + '\'s ', sent)    
        messages.append(re.sub('.\n','',sent))
        ids.append(identifier[docid])

    
    sent_embeddings = [*map(lambda sent:(session.run(embedded_text, feed_dict={text_input: [sent]})), messages)]
    query_embedding = session.run(embedded_text, feed_dict={text_input: [query]})

    # Sentences re-ranking using embedding
    similarities = [*map(lambda sent_embedding:(cosine_similarity(sent_embedding, query_embedding).flatten()), sent_embeddings)]
    sorted_pair = sort(list(zip([list(i)[0] for i in similarities], list(range(len(messages))))))
    
    #evidence = []
    unlabeled_sents_ids = []
    sim = []
    unlabeled_sents = []
    output = []
    for score,index in sorted_pair[:k]:
        #evidence.append(sent_embeddings[index])
        unlabeled_sents_ids.append(ids[index])
        sim.append(score)
        unlabeled_sents.append(messages[index])
        
    scores = [*map(lambda sent:(get_label_scores(query, sent)), unlabeled_sents)]

    output = [] 
    i = 0
    for sup, ref, noI in scores:
        output.append((sim[i], sup, ref, noI), (unlabeled_sents_ids[i]))
        i += 1

    return output

In [ ]:
import json
with open('test-unlabelled.json') as f:
    test = json.load(f)

In [ ]:
%%time
test_sents= dict()
for i in devset:
    query = test[i]['claim']
    output = label_and_evidenve(query, 5)
    test[query] = output
    count += 1

In [ ]:
predictions = defaultdict(list)
for i in test_sents:
    results = []
    ids = []
    for sents in test_sents[i]:
        for sent in sents:
            ids.append(sent[1])
            result = list(model.predict(pd.DataFrame(list(sent[0])).T).flatten())
            results.append(result)
    predictions[i] = [results, ids]

In [756]:
# Reference: https://github.com/uclmr/fever/blob/master/jack_reader.py
def aggregate_preds(predictions, top_one_sent=False):
    """return the most popular label
    """
    vote = dict()
    preds = []
    for pred in predictions[0]:
        preds.append(["SUPPORTS", "NOT ENOUGH INFO", "REFUTES"][np.argmax(pred)])

    for pred in preds:
        if pred not in vote:
            vote[pred] = 1
        else:
            vote[pred] += 1
    
    supports = "SUPPORTS"
    refutes = "REFUTES"
    nei = "NOT ENOUGH INFO"
    
    # believe more-likely evidence if both supports and refutes appears in the pred_list
    if supports in vote and refutes in vote:
        for pred in preds:
            if pred in [supports, refutes]:
                final_verdict = pred
                break
    elif supports in vote:
        final_verdict = supports
    elif refutes in vote:
        final_verdict = refutes
    else:
        final_verdict = nei
    if top_one_sent:
        final_verdict = preds[0]

     

    if final_verdict != nei:
        for temp in predictions[1]:
            page_id, sent_num = temp
            try:
                evidence = (page_id, int(sent_num))
                evidences.append(evidence)
            except:
                    # Some values of sent_nums are string and could not be 
                    # transfer to integer, we will store the original value.
                print("Obammmmmaaa")
                evidences.append(temp)
                continue

    return (final_verdict, evidences)

In [ ]:
%%time
final_labels_and_evidences = dict()
for claim in predictions:
    temp = predictions[claim]
    final_labels_and_evidences[claim] = aggregate_preds(temp)

In [ ]:
for i in test:
    if test[i]["claim"] in final_labels_and_evidences.keys():
        test[i]["label"] = final_labels_and_evidences[test[i]["claim"]][0]
        test[i]["evidence"] = final_labels_and_evidences[test[i]["claim"]][1]
    else:
        test[i]["label"] = "NOT ENOUGH INFO"
        test[i]["evidence"] = []

In [ ]:
with open('testOutput.json', 'w') as fp:
    json.dump(test, fp)

# Appendix: Parellized entaiment function

In [ ]:
import ray
@ray.remote
def get_label_probs2(h,p):
    return predictor.predict(h,p)['label_probs']
def get_label_probs1(hypothesis,premises):
    ray.init(num_cpus=2,num_gpus=1,ignore_reinit_error=True)
    temp_results = ray.get([get_label_probs2.remote(hypothesis,premise) for premise in premises])
    return temp_results
#Chika's processed test sentences
test_processed = None
hypothesis_evidences = {}
with open("test_sents.pkl",'rb') as f:
    test_processed = pickle.load(f)
for key,value in test_processed.items():
    temp = []
    #print(key)
    for tuple_3 in value:
        temp.append(tuple_3[0])
        hypothesis_evidences[key]=temp
    #print("----------------------------------")
    
    
st=time.time()
final_results = []
for hypothesis,evidences in hypothesis_evidences.items():
    final_results.append(get_label_probs1(hypothesis,evidences))
print((time.time()-st)/60,"minutes")
#TO store {claim:allennlpscores}
claim_allennlpscores_900 = {}
for index,hyp_evis in enumerate(hypothesis_evidences.items()):
    #print(len(hyp_evis[1])==len(final_results[index]))
    key = hyp_evis[0]#THE CLAIM
    value = final_results[index]
    claim_allennlpscores[key] = value
with open('claim_allennlpscores.pkl', 'wb') as output:
    pickle.dump(claim_allennlpscores, output, pickle.HIGHEST_PROTOCOL)